In [1]:
# Llm can t take actions , Agents use LLM as reasoning to determine the action and input to pass tp them. Post Agent execution , result i sfed back into the LLM to detrmine the next step

In [ ]:
# ! pip install langgraph

### Search engine Agent

In following this tutorial, you will learn how to:

1. Use language models, in particular their tool calling ability
2. Use a Search Tool to look up information from the Internet
3. Compose a LangGraph Agent, which use an LLM to determine actions and then execute them
4. Debug and trace your application using LangSmith

In [3]:
from dotenv import load_dotenv
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
load_dotenv()

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-3.5-turbo')

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is the weather in Chicago")
print(search_results)
tools = [search]

[{'url': 'https://world-weather.info/forecast/usa/chicago/june-2024/', 'content': 'Extended weather forecast in Chicago. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ Chicago Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Chicago', 'region': 'Illinois', 'country': 'United States of America', 'lat': 41.85, 'lon': -87.65, 'tz_id': 'America/Chicago', 'localtime_epoch': 1718733992, 'localtime': '2024-06-18 13:06'}, 'current': {'last_updated_epoch': 1718733600, 'last_updated': '2024-06-18 13:00', 'temp_c': 29.2, 'temp_f': 84.5, 'is_day': 1, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'wind_mph': 15.2, 'wind_kph': 24.5, 'wind_degree': 192, 'wind_dir': 'SSW', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.07, 'precip_in': 0.0, 'humidity': 56, 'cloud': 55, 'feelslike_c': 31.2, 

In [7]:
# we use .bind_tools to give LLM the knowledge of the tools - 
model_with_tools = model.bind_tools(tools)
model_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002875DEED910>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002875DEFE490>, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}}}]})

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])
print("Content res :" , response.content)
print("Tool Calls  :" , response.tool_calls)

Content res : Hello! How can I assist you today?
Tool Calls  : []


In [12]:
response = model_with_tools.invoke([HumanMessage(content="Whats weather in Chicago!")])
print("Content res :" , {response.content})
print("Tool Calls  :" , response.tool_calls)

Content res : {''}
Tool Calls  : [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Chicago'}, 'id': 'call_yZOMgA83RsmEoI1z7aZMaLpy'}]


In [13]:
# content is null , but there is a tool call 
# this isn't calling the tool yet , its just telling us to . Inorder to actually call it , we would need an agent

### create an Agent

Will use `LangGraph` to construct the agent. Currently we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [16]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model, tools)

### Run the agent

In [17]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
response["messages"]

[HumanMessage(content='hi!', id='33e35669-4ea5-471b-8522-b4d48a8bef90'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aa41ec5c-a903-4599-9a0b-776a06322a89-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [20]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Chicago?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Chicago?', id='f2c0c81a-9528-4f0d-8bd2-e4c00a17a34e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_irDSBQ1XkB9rr8wvXkUuLnAp', 'function': {'arguments': '{"query":"weather in Chicago"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 88, 'total_tokens': 108}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-efc1c759-fdee-41e1-9677-6045419bfaa9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Chicago'}, 'id': 'call_irDSBQ1XkB9rr8wvXkUuLnAp'}], usage_metadata={'input_tokens': 88, 'output_tokens': 20, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.chicagotribune.com/2024/06/18/tuesday-forecast-temperatures-in-low-90s-isolated-thunderstorms-possible/", "content": "National Weather Service officials are expecti

### Streaming Messages

We've seen how the agent can be called with `.invoke` to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Chicago?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vRyCFxeHeJvtOU80gRJ2dgBr', 'function': {'arguments': '{"query":"weather in Chicago"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 88, 'total_tokens': 108}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e06e0052-d4b3-45b6-a14c-3e423289767a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Chicago'}, 'id': 'call_vRyCFxeHeJvtOU80gRJ2dgBr'}], usage_metadata={'input_tokens': 88, 'output_tokens': 20, 'total_tokens': 108})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.wunderground.com/hourly/us/il/chicago/60622/date/2024-6-18", "content": "Chicago Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather con

### Streaming Tokens can be benificial if we need to pass it to other info , we use `.astream_events`

In [24]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Chicago?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in Chicago'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.accuweather.com/en/us/chicago/60608/june-weather/348308', 'content': 'Get the monthly weather forecast for Chicago, IL, including daily high/low, historical averages, to help you plan ahead.'}, {'url': 'https://world-weather.info/forecast/usa/chicago/june-2024/', 'content': 'Extended weather forecast in Chicago. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ Chicago Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}]
--
I| found| some| resources| for| the| weather| in| Chicago|:

|1|.| [|Acc|u|Weather|](|https|://|www|.acc|u|weather|.com|/en|/us|/ch|icago|/|606|08|/j|une|-|weather|/|348|308|):| Get| the| monthly| weather| forecast| for| Chicago|,| IL|,| including| daily| high|/|low|,| historical| averages|,| to| help| you| plan| ahead|.
|   
|2|.| [|World| Weather|](|h

### Addding in Memory

this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [25]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4a2b1f87-81b3-4bfa-b9e8-fa26db42e211-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you further, Bob?', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 108, 'total_tokens': 123}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d555dc16-cd5c-44e9-affc-b06db4c00057-0', usage_metadata={'input_tokens': 108, 'output_tokens': 15, 'total_tokens': 123})]}}
----
